In [18]:
DATASET_DIR = 'dataset'
LABELS = {
    'обеспечение исполнения контракта': 0,
    'обеспечение гарантийных обязательств': 1
}

In [20]:
import json
import pandas as pd

In [10]:
with open(f'{DATASET_DIR}/train.json', 'rb') as f:
    train = json.load(f)

In [16]:
len(train)

1799

In [17]:
train[1798]

{'id': 926039184,
 'text': 'Государственная корпорацИя по атомной энергии «росатом» акционерное общество «НИИЭФА им. Д.В. Ефремова» (АО «НИИЭФА») УТВЕРЖДАЮ: Генеральный директор АО «НИИЭФА» ____________________ Е.А. Сакадынец «____» ______________ 2022г. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК (ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ) открытый запрос котировок в электронной форме на право заключения договора на поставку переносного рентгеновского аппарата направленного действия ICM SITE-X D1802 Извещение размещено на официальных сайтах http://www.zakupki.gov.ru, http://zakupki.rosatom.ru и http://roseltorg.ru/ от «07» сентября 2022 года № 220907/0459/___ Санкт-Петербург, 2022г. СОДЕРЖАНИЕ ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК 1 1. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАКУПКИ 3 ЧАСТЬ 1 9 2. ТРЕБОВАНИЯ. ДОКУМЕНТЫ. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ В ЗАКУПКЕ. 9 2.1. ТРЕБОВАНИЯ. ДОКУМЕНТЫ, ПОДТВЕРЖДАЮЩИЕ СООТВЕТСТВИЕ УСТАНОВЛЕННЫМ ТРЕБОВАНИЯМ. 9 2.1.1. Требования к участникам закупки 9 2.1.2. Требования к продукции 12 2.2. 

In [19]:
train[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [ ]:
train_df = pd.DataFrame()